In [1]:
import glob
import os
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np

import data
import torch
from models import imagebind_model
from models.imagebind_model import ModalityType

c:\miniconda3\envs\imagebind\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\miniconda3\envs\imagebind\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module i

In [17]:
imgs = glob.glob('data/da_imgs/**/*.png', recursive=True)

# Iterate over the image paths
for img in tqdm(imgs):
    # Try to open the image
    try:
        image = Image.open(img)
    except OSError:
        # If the image is corrupted, delete it
        os.remove(img)

imgs = glob.glob('data/da_imgs/**/*.png', recursive=True)

100%|██████████| 9865/9865 [00:01<00:00, 6979.13it/s]


In [2]:
img_path = "MM_DA_paintings/*"
imgs = glob.glob(img_path, recursive=True)

In [ ]:
# Instantiate model
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

In [4]:
# load data
inputs = {
    ModalityType.VISION: data.load_and_transform_vision_data(imgs, device)
}

In [5]:
# get embeddings
with torch.no_grad():
    embeddings = model(inputs)

In [24]:
# get embeddings using multiple modalities
print(torch.softmax(embeddings[ModalityType.VISION], dim=1))

torch.Size([10, 1024])


In [13]:
# vision embeddings
embeddings['vision'][0]

tensor([ 0.0054,  0.0031, -0.0132,  ...,  0.0475,  0.0046,  0.0109])

In [6]:
np_embeddings = [emb.cpu().numpy() for emb in embeddings['vision']]

In [9]:
os.path.basename(imgs[0])

'1.jpg'

In [10]:
dest_path = "embeddings"

for i in range(0,len(imgs)):
    file_path = os.path.join(dest_path, f"{os.path.basename(imgs[i])}.npy")
    np.save(file_path, np_embeddings[i])